In [1]:
import pickle
import numpy as np
from random import shuffle
import agent_code.my_agent.algorithms as alg
from settings import s
import time
import matplotlib.pyplot as plt
import copy

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
weights = np.load('./agent_code/my_agent/models/weights_initx.npy')
weights

array([ -2.06550326,   8.76784424,  -7.        ,  -8.53001744,
       -12.14029016,   1.5       ,  -0.73029129,  -7.93449674,
       -17.8637092 ,  -2.02344178])

In [2]:
file = open('C:/Users/Lily/Documents/UNI/FML/project/local_branch/data-collection/75-crate-no-opponents.pickle', 'rb')
data = pickle.load(file)
data = sorted(data, key=lambda k: k['state']['step'])